In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tqdm import tqdm
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

pd.options.display.max_columns = None

/home/georgebelsky/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/georgebelsky/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/georgebelsky/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/georgebelsky/.local/lib/python3.7/site-packages/tensorflow/python/f

In [2]:
data = pd.read_table("data.txt", sep=';')
data['VEHICLE'] = data['VEHICLE_MAKE'] + " " + data['VEHICLE_MODEL']
data = data.drop(['VEHICLE_MAKE', 'VEHICLE_MODEL', ], axis=1)

In [3]:
# Препроцессинг данных
all_columns = data.columns.to_series().groupby(data.dtypes).groups
object_columns = list(all_columns[list(all_columns.keys())[2]])
for col in object_columns:
    qur_le = LabelEncoder()
    qur_le.fit(data[col])
    data[col] = qur_le.transform(data[col])
    globals()[col + "_LE"] = qur_le #создаем переменную с уникальным именем и записываем в нее LabelEncouder текущего столбца

train_data = data[data['DATA_TYPE'] == 1]
y = train_data['POLICY_IS_RENEWED']
X = train_data.drop(['POLICY_IS_RENEWED', 'DATA_TYPE', 'POLICY_ID'], axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

test_data = data[data['DATA_TYPE'] == 0]
X_test = test_data.drop(['POLICY_IS_RENEWED', 'DATA_TYPE', 'POLICY_ID'], axis=1)

In [4]:
# Матрица корреляций признаков
corr = train_data.corr()
corr.style.background_gradient(cmap='coolwarm')

/home/georgebelsky/anaconda3/lib/python3.7/site-packages/pandas/io/formats/style.py:1089: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
/home/georgebelsky/anaconda3/lib/python3.7/site-packages/pandas/io/formats/style.py:1090: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax
/home/georgebelsky/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,DATA_TYPE,POLICY_ID,POLICY_BEGIN_MONTH,POLICY_END_MONTH,POLICY_IS_RENEWED,POLICY_SALES_CHANNEL,POLICY_SALES_CHANNEL_GROUP,POLICY_BRANCH,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_ENGINE_POWER,VEHICLE_IN_CREDIT,VEHICLE_SUM_INSURED,POLICY_INTERMEDIARY,INSURER_GENDER,POLICY_CLM_N,POLICY_CLM_GLT_N,POLICY_PRV_CLM_N,POLICY_PRV_CLM_GLT_N,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,CLAIM_AVG_ACC_ST_PRD,POLICY_HAS_COMPLAINTS,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,CLIENT_REGISTRATION_REGION,POLICY_PRICE_CHANGE,VEHICLE
DATA_TYPE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
POLICY_ID,nan,1.000000,0.010113,0.010290,-0.017961,-0.001358,-0.002904,0.001243,0.003593,-0.000107,-0.005388,-0.003122,-0.004391,0.003163,0.003085,0.010127,0.008997,0.002944,0.002480,0.004608,0.003908,0.004245,0.001174,0.003415,-0.000824,-0.000858,0.000982,-0.003614,-0.001980
POLICY_BEGIN_MONTH,nan,0.010113,1.000000,0.998021,-0.044136,0.005104,0.017006,0.038909,-0.007664,0.021788,0.020635,0.072389,0.054739,-0.107931,0.000550,0.038237,0.035194,0.163441,0.164514,-0.016303,-0.009333,-0.003095,0.020288,0.004211,-0.102653,-0.058950,0.005365,0.004012,-0.023081
POLICY_END_MONTH,nan,0.010290,0.998021,1.000000,-0.044765,0.005353,0.016944,0.038345,-0.007609,0.022163,0.021073,0.072935,0.055490,-0.108916,0.000508,0.038388,0.035371,0.164910,0.166007,-0.017225,-0.009956,-0.002890,0.020456,0.004539,-0.103895,-0.058941,0.005016,0.004045,-0.022890
POLICY_IS_RENEWED,nan,-0.017961,-0.044136,-0.044765,1.000000,-0.030981,-0.017969,0.085185,0.027025,-0.024898,-0.008983,-0.048445,-0.037684,0.052050,-0.012823,-0.154643,-0.172653,-0.141845,-0.143015,0.085824,0.096507,-0.034504,-0.046212,-0.021662,0.107898,-0.000358,0.089686,-0.070610,0.008368
POLICY_SALES_CHANNEL,nan,-0.001358,0.005104,0.005353,-0.030981,1.000000,0.685215,-0.132213,0.029112,0.011246,-0.078189,0.131166,-0.027227,0.113135,0.005560,0.019035,0.019087,0.047513,0.045654,-0.076931,-0.147400,0.002412,0.008284,0.005070,-0.115817,-0.026049,-0.085668,-0.045921,-0.046238
POLICY_SALES_CHANNEL_GROUP,nan,-0.002904,0.017006,0.016944,-0.017969,0.685215,1.000000,-0.092652,0.018187,0.021679,-0.017805,0.174236,0.047863,-0.010130,-0.013507,0.051760,0.049443,0.047314,0.043934,-0.065118,-0.170119,-0.005312,0.010667,-0.000384,-0.115236,-0.074580,-0.050061,-0.065972,-0.062823
POLICY_BRANCH,nan,0.001243,0.038909,0.038345,0.085185,-0.132213,-0.092652,1.000000,0.053571,0.004945,-0.122278,0.062355,-0.120194,-0.054463,0.011455,-0.025381,-0.027774,-0.052196,-0.052796,-0.017590,0.098645,-0.013270,-0.002309,-0.007328,0.049053,0.048248,0.612091,-0.000878,0.044832
POLICY_MIN_AGE,nan,0.003593,-0.007664,-0.007609,0.027025,0.029112,0.018187,0.053571,1.000000,0.025250,-0.063090,-0.075981,-0.047904,-0.115586,0.087080,-0.040879,-0.034466,-0.021221,-0.018276,-0.092059,-0.023222,-0.017234,-0.026565,-0.021394,0.026284,-0.155315,0.044446,-0.019457,0.091891
POLICY_MIN_DRIVING_EXPERIENCE,nan,-0.000107,0.021788,0.022163,-0.024898,0.011246,0.021679,0.004945,0.025250,1.000000,-0.000683,0.033629,0.048011,-0.015654,-0.000608,-0.002888,-0.000920,0.130298,0.131931,-0.038196,-0.013580,-0.003585,-0.007566,-0.000459,-0.091839,0.042772,0.001226,-0.008690,-0.009157


In [5]:
# Градиентный бустинг на решающих деревъях (лучшая модель по результативности)
gbc = GradientBoostingClassifier(n_estimators=1000)
gbc.fit(X_train,y_train)
pred_train = gbc.predict(X_train)
pred_val = gbc.predict(X_val)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))

0.7472264836495761
0.7211600568402016


In [6]:
# запись результата в файл result.csv
result = pd.DataFrame(test_data['POLICY_ID'])
result['POLICY_IS_RENEWED'] = gbc.predict(X_test)
pred_proba_total = gbc.predict_proba(X_test)
a = list(pd.DataFrame(pred_proba_total)[1])
result['POLICY_IS_RENEWED_PROBABILITY'] = a
result.to_csv('result.csv', index=False)
display(result)

,POLICY_ID,POLICY_IS_RENEWED,POLICY_IS_RENEWED_PROBABILITY
9,10,1,0.851044
11,12,1,0.761780
12,13,1,0.599174
21,22,0,0.422448
36,37,1,0.824979
...,...,...,...
96573,96574,1,0.865523
96576,96577,1,0.558682
96585,96586,1,0.899286
96594,96595,1,0.807645


In [7]:
# ----------- Ниже приведены прочие варианты моделей -------------------

In [8]:
# Случайный лес
random_forest = RandomForestClassifier(
    n_estimators=1000,
    n_jobs=-1,
    max_features=int(np.sqrt(26)),
    max_depth=9)
random_forest.fit(X_train,y_train)
pred_train = random_forest.predict(X_train)
pred_val = random_forest.predict(X_val)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))

0.7276705692369803
0.7024932179304999


In [9]:
# Решающие деревья
tree = DecisionTreeClassifier(criterion = "gini", max_depth=30, max_leaf_nodes=50)
tree.fit(X_train,y_train)
pred_train = tree.predict(X_train)
pred_val = tree.predict(X_val)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))

0.7042067016552281
0.7019764888257332


In [11]:
# Адаптивный бустинг
abc = AdaBoostClassifier() #(algorithm='SAMME')
abc.fit(X_train,y_train)
pred_train = abc.predict(X_train)
pred_val = abc.predict(X_val)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))

0.6974727492935002
0.697584291435215


In [12]:
# Логистическая регрессия
lr = LogisticRegression(C=0.001, penalty='l1', solver='liblinear', max_iter=5000)
lr.fit(X_train,y_train)
pred_train = lr.predict(X_train)
pred_val = lr.predict(X_val)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))

0.6528542591844974
0.6484950264823666


In [10]:
# Беггинг градиентного бустинга (долго считается)
gbc = GradientBoostingClassifier(n_estimators=1000)
bc = BaggingClassifier(
    gbc,
    n_estimators=10,
    n_jobs=-1,
    random_state=42)
bc.fit(X_train,y_train)
pred_train = bc.predict(X_train)
pred_val = bc.predict(X_val)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))

0.7455147355672184
0.7230331998449813


In [13]:
# Стекинг серии алгоритмов (долго считается)
cover_train, cover_test = train_test_split(train_data, test_size=0.2)
cover_X_train, cover_y_train = cover_train.drop(['POLICY_IS_RENEWED', 'DATA_TYPE', 'POLICY_ID'], axis=1), cover_train['POLICY_IS_RENEWED']
cover_X_test, cover_y_test = cover_test.drop(['POLICY_IS_RENEWED', 'DATA_TYPE', 'POLICY_ID'], axis=1), cover_test['POLICY_IS_RENEWED']

scaler = StandardScaler()
cover_X_train = scaler.fit_transform(cover_X_train)
cover_X_test = scaler.transform(cover_X_test)

def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

def generate_metafeatures(classifiers, X_train, X_test, y_train, cv):
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

cv = KFold(n_splits=5, shuffle=True)

stacked_features_train, stacked_features_test = generate_metafeatures([
    LogisticRegression(C=0.001, penalty='l1', solver='liblinear', max_iter=5000),
    AdaBoostClassifier(),
    DecisionTreeClassifier(criterion = "gini", max_depth=30, max_leaf_nodes=50),
    RandomForestClassifier(n_estimators=1000, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=1000)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

total_features_train = np.hstack([cover_X_train, stacked_features_train])
total_features_test = np.hstack([cover_X_test, stacked_features_test])

np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs')
clf.fit(stacked_features_train, cover_y_train)
accuracy_score(clf.predict(stacked_features_test), cover_y_test)

100%|██████████| 5/5 [13:37<00:00, 163.53s/it]


0.7215476036687767

In [26]:
# нейросеть (простая на 3 Dense слоях)
X_numpy = np.array(X)
X_numpy = np.expand_dims(X_numpy, axis=1)
y_numpy = np.array(y)
y_numpy = np.expand_dims(y_numpy, axis=1)
X_train, X_val, y_train, y_val = train_test_split(X_numpy, y_numpy, test_size=0.2, random_state=42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape = [None, 26]),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid'),
])
opt = tf.keras.optimizers.Adam(learning_rate= 0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss = 'binary_crossentropy',optimizer = opt, metrics = ['accuracy'])
model.summary()

num_epochs = 10
model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val))

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, None, 256)         6912      
_________________________________________________________________
dense_49 (Dense)             (None, None, 256)         65792     
_________________________________________________________________
dense_50 (Dense)             (None, None, 512)         131584    
_________________________________________________________________
dense_51 (Dense)             (None, None, 2)           1026      
Total params: 205,314
Trainable params: 205,314
Non-trainable params: 0
_________________________________________________________________
Train on 61925 samples, validate on 15482 samples
Epoch 1/10
61925/61925 [==============================] - 5s 89us/sample - loss: 325.6375 - acc: 0.6227 - val_loss: 0.6633 - val_acc: 0.6216
Epoch 2/10
61925/61925 [==============================